# Applied Data Science Capstone-Week 3 Peer graded Assignment

## In this notebook I will be segmenting Toronto neighborhoods. 

### 1. Building the dataframe

The first thing to do is to import the necessary libraries in order this notebook to work.

In [1]:
    from bs4 import BeautifulSoup
    import requests
    import numpy as np
    import pandas as pd

The next step is to get our information from the source i.e. https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M  , to do that we will use the reqeuests library.

In [2]:
toronto_html=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(toronto_html,'lxml')
#print(soup.prettify())
table = soup.table#find('table', class_='wikitable sortable')
#print(table.prettify())

Here we have created a soup object that contains the text of the html page. Then, we extract the table class from the soup object. From this, we can identify the table rows and columns as follows:

In [3]:
column_names=['PostalCode','Borough','Neighborhood']
df=pd.DataFrame(columns=column_names)
for row in table.find_all('tr'):
    #print(row.prettify())
    row_text=row.text
    column=row_text.split('\n')
    PostalCode=column[1]
    Borough=column[2]
    Neighborhood=column[3]
    df=df.append({'PostalCode':PostalCode,'Borough':Borough,'Neighborhood':Neighborhood}, ignore_index=True)

The columns text is appended to the created dataframe 'df' in each iteration. This gives us the following:

In [4]:
df.head()

,PostalCode,Borough,Neighborhood
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


As we can see, we need to drop the first row, as we have created our own column names.

In [5]:
df.drop(0, inplace=True)
df.reset_index(drop=True,inplace=True)

The next step is to drop the 'Not assigned' values in the 'Borough' column. To do so, we first replace the 'Not assigned' to np.nan and then used the method .dropna in our data frame.

In [6]:
df.replace('Not assigned', np.nan, inplace=True)
df.reset_index(drop=True,inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [7]:
df.dropna(subset=['Borough'],axis=0,inplace=True)
df.reset_index(drop=True,inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Lastly, we can manually replace the NaN value in the 'Neighborhood' column:

In [8]:
df['Neighborhood'].replace(np.nan, 'Queen\'s Park',inplace=True)

The last step is to group the postal codes and combine the Neighborhood names into the same row.

In [9]:
df=df.groupby(['PostalCode','Borough'], as_index=False,sort = True).agg(lambda x: ','.join(x))
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
df.shape

(103, 3)

Now we can import the geospatial data for the neighborhoods of Toroto usingn the read_csv method from pandas.

In [11]:
df1=pd.read_csv('http://cocl.us/Geospatial_data')
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Then, the data is appended to our main data frame.

In [12]:
df['Latitude']=df1['Latitude']
df['Longitude']=df1['Longitude']
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### 2. Getting the geospatial data

To begin with the segmentation of the neighborhoods in Toronto, first it is needed to install and import the requiered libraries.

In [13]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



Then, the geolocation data from the city is adquired to create the map using the folium libraries.

In [14]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 3. Connecting to the Foursquare API and exploring neighborhoods in Toronto

In this point, we will be introducing the Foursquare API's credentials to get information about the neighborhoods in order to segment and then cluster them.

In [16]:
CLIENT_ID = '1C1JD0PTGEJXCUTXMPR3GL5QGCWLJ041J302NEVM2H2RKV3X' # your Foursquare ID
CLIENT_SECRET = 'MUEXEC115P3ZED2MSRCNPF0XWCZFXWPOV2WNGHBPA1DEYA1V' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1C1JD0PTGEJXCUTXMPR3GL5QGCWLJ041J302NEVM2H2RKV3X
CLIENT_SECRET:MUEXEC115P3ZED2MSRCNPF0XWCZFXWPOV2WNGHBPA1DEYA1V


The function defined bellow gives us the venues for all the neighborhoods in any city we want, in our case, Toronto:

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Let's call the function and save the venues in a dataframe we will call toronto_venues:

In [18]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'])

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [19]:
print(toronto_venues.shape)
toronto_venues.head()

(2247, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


Let's see how many venues per neighborhood there are:

In [20]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",8,8,8,8,8,8
"Alderwood,Long Branch",9,9,9,9,9,9
"Bathurst Manor,Downsview North,Wilson Heights",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",26,26,26,26,26,26
Berczy Park,56,56,56,56,56,56


As we can see from above, there exists some neighbors with more venue counts than other, may be related with higher life expectations or dense urban neighbors. Let´s now analize each neighbor, for that, we can first check how many unique categories exists:

In [21]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 278 uniques categories.


To star with the clustering method, we have to convert these unique categories in one-hot encodings in order to the method can clasiffy them:

In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

toronto_onehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.0,0.000000,0.000000,0.000000
1,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,"Alderwood,Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
5,"Bathurst Manor,Downsview North,Wilson Heights",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
6,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
7,"Bedford Park,Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
8,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
9,"Birch Cliff,Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


Let's print each neighborhood along with the top 5 most common venues

In [24]:
num_top_venues = 5
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3  American Restaurant  0.04
4           Steakhouse  0.04


----Agincourt----
                venue  freq
0      Breakfast Spot   0.2
1              Lounge   0.2
2  Chinese Restaurant   0.2
3      Sandwich Place   0.2
4        Skating Rink   0.2


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                       venue  freq
0                 Playground   0.5
1                       Park   0.5
2  Middle Eastern Restaurant   0.0
3              Movie Theater   0.0
4                      Motel   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store  0.25
1              Pharmacy  0.12
2  Fast Food Restaurant  0.12
3           Pizza Place  0.12
4   Fried Chicken Joint  0.12


----Alderwood,Long Branch----
        

We can put that information in a dataframe by defining the function bellow:

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,American Restaurant,Steakhouse,Hotel,Sushi Restaurant,Restaurant,Bakery,Burger Joint
1,Agincourt,Lounge,Chinese Restaurant,Sandwich Place,Breakfast Spot,Skating Rink,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Playground,Park,Yoga Studio,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Pizza Place,Sandwich Place,Pharmacy,Airport Lounge,Department Store,Ethiopian Restaurant
4,"Alderwood,Long Branch",Pizza Place,Pharmacy,Skating Rink,Dance Studio,Coffee Shop,Pub,Sandwich Place,Gym,Airport Service,Deli / Bodega


### 4. Clustering the neighborhoods

We choose to cluster the neighborhoods using k-means clustering method with k=5

In [58]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 4, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

Then we add this labels to our dataframe:

In [60]:
neighborhoods_venues_sorted.drop('Cluster Labels',axis=1, inplace=True) #Delete the column if already exists
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,2.0,Golf Course,Bar,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,2.0,Spa,Pizza Place,Electronics Store,Intersection,Breakfast Spot,Rental Car Location,Medical Center,Mexican Restaurant,Yoga Studio,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2.0,Coffee Shop,Korean Restaurant,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Hakka Restaurant,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Convenience Store,Dim Sum Restaurant,Empanada Restaurant


As we can see, the cluster labels in the dataframe avobe are in type float. This will make difficult to visualize the clusters in a map. To solve this, we have to first drop the clusters with no labels on it, and second convert the cluster labels column to type int.

In [61]:
toronto_merged.dropna(subset=['Cluster Labels'],axis=0, inplace=True)
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype('int')

We can now create a map to visualize the resulting clusters:

In [62]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 5. Exploring each cluster

Let's explore each cluster deeply.

#### Cluster #1

This cluster may be  suburbs

In [74]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,"Humber Bay,King's Mill Park,Kingsway Park Sout...",0,Baseball Field,Construction & Landscaping,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
97,"Emery,Humberlea",0,Baseball Field,Construction & Landscaping,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop


#### Cluster #2

This cluster may be a family ideal

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough Village,1,Pizza Place,Playground,Dive Bar,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
81,"The Junction North,Runnymede",1,Pizza Place,Bus Line,Caribbean Restaurant,Convenience Store,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant
95,"Bloordale Gardens,Eringate,Markland Wood,Old B...",1,Pharmacy,Beer Store,Pizza Place,Convenience Store,Café,Liquor Store,Gift Shop,Electronics Store,Dumpling Restaurant,Drugstore
96,Humber Summit,1,Pizza Place,Empanada Restaurant,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant


#### Cluster #3

This cluster corresponds to neighbors with more sports facilities 

In [73]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge,Malvern",2,Fast Food Restaurant,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop
1,"Highland Creek,Rouge Hill,Port Union",2,Golf Course,Bar,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant
2,"Guildwood,Morningside,West Hill",2,Spa,Pizza Place,Electronics Store,Intersection,Breakfast Spot,Rental Car Location,Medical Center,Mexican Restaurant,Yoga Studio,Discount Store
3,Woburn,2,Coffee Shop,Korean Restaurant,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
4,Cedarbrae,2,Hakka Restaurant,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Convenience Store,Dim Sum Restaurant,Empanada Restaurant
6,"East Birchmount Park,Ionview,Kennedy Park",2,Discount Store,Chinese Restaurant,Bus Station,Coffee Shop,Hobby Shop,Convenience Store,Department Store,Yoga Studio,Dim Sum Restaurant,Diner
7,"Clairlea,Golden Mile,Oakridge",2,Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant,Bus Station,Dog Run,Diner,Discount Store,Dive Bar
8,"Cliffcrest,Cliffside,Scarborough Village West",2,Movie Theater,Motel,American Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Yoga Studio
9,"Birch Cliff,Cliffside West",2,Café,General Entertainment,College Stadium,Skating Rink,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
10,"Dorset Park,Scarborough Town Centre,Wexford He...",2,Indian Restaurant,Pet Store,Chinese Restaurant,Light Rail Station,Latin American Restaurant,Vietnamese Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner


#### Cluster #4

This cluster corresponds to negihbors with dense concentration of banks office

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,"Cloverdale,Islington,Martin Grove,Princess Gar...",3,Bank,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Department Store


#### Cluster #5

This cluster corresponds to neghbors with parks and convinience facilities.

In [72]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Agincourt North,L'Amoreaux East,Milliken,Steel...",4,Playground,Park,Yoga Studio,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
23,York Mills West,4,Park,Bank,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant
25,Parkwoods,4,Park,Fast Food Restaurant,Food & Drink Shop,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
30,"CFB Toronto,Downsview East",4,Park,Other Repair Shop,Airport,Bus Stop,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
40,East Toronto,4,Park,Convenience Store,Metro Station,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Yoga Studio
44,Lawrence Park,4,Park,Bus Line,Swim School,Yoga Studio,Dive Bar,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
50,Rosedale,4,Park,Trail,Playground,Yoga Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
74,Caledonia-Fairbanks,4,Park,Pharmacy,Women's Store,Market,Fast Food Restaurant,Dive Bar,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
79,"Downsview,North Park,Upwood Park",4,Park,Bakery,Construction & Landscaping,Basketball Court,Yoga Studio,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run
90,"The Kingsway,Montgomery Road,Old Mill North",4,Park,River,Pool,Yoga Studio,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
